In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 800
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 20
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)
beta[3] = change_type(beta_ref, 3)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[5.96433002 5.17173002 5.96433002]


In [5]:
cp_truth

array([ 800, 1600, 2400])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 3200, 20)

In [8]:
np.random.seed(0)

m_intervals = 50
grid_n = 200
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_wbs = np.zeros(B)
loc_error_wbs = np.zeros(B)
K_wbs = np.zeros(B)

cp_best_list = []
param_best_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    wbs_fit = wbs_cv_bt(m_intervals, grid_n, lam_list, gamma_list, smooth = 5, buffer = 5)
    res = wbs_fit.fit((X_train, Y_train), (X_test, Y_test))
    cp_best, cp_val, cusum_val, threshold_best, grid = res   
    run_time_wbs[b] = time.time() - start_time
    loc_error_wbs[b] = cp_distance(cp_best, cp_truth)
    K_wbs[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(threshold_best)

    print(b)

print('---------- wbs -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_wbs.mean(), run_time_wbs.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_wbs.std(), run_time_wbs.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_wbs < T - 1), sum(K_wbs == T - 1), sum(K_wbs > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- wbs -----------
avg loc error: 240.48, avg time: 144.20785520553588
std loc error: 220.3177922910449, std time: 12.5031040774292
K < K*: 0, K = K*: 40, K > K*: 60


In [9]:
loc_error_wbs

array([  0.,   0., 336., 256.,   0., 384.,   0.,  16., 464., 512., 576.,
         0.,   0.,   0., 464.,   0., 144., 304., 400., 304., 352., 624.,
       368., 528., 336., 240.,   0., 480., 496.,   0., 672., 256.,  32.,
       288.,   0., 544., 384.,   0.,   0., 592.,  16., 432., 320., 384.,
         0., 224.,   0., 288.,   0., 352., 160.,  16., 624.,  16.,   0.,
       560., 528.,   0.,  16., 608.,   0., 576., 160., 176., 224.,   0.,
         0.,   0., 640.,   0., 432., 384.,   0., 256.,  16.,   0., 336.,
       336., 352., 304., 304.,   0.,   0.,  16., 304.,   0.,   0., 512.,
       384.,   0., 352., 272., 384., 688.,   0., 480.,   0., 512., 352.,
       400.])

In [10]:
cp_best_list

[[800, 1600, 2400],
 [800, 1600, 2400],
 [800, 1296, 1600, 2064, 2400],
 [800, 1344, 1600, 2400],
 [800, 1600, 2400],
 [800, 1600, 2016, 2176, 2400],
 [800, 1600, 2400],
 [800, 1616, 2400],
 [800, 1600, 2400, 2864],
 [576, 800, 1600, 2400, 2912],
 [800, 1616, 2048, 2400, 2976],
 [800, 1600, 2400],
 [800, 1600, 2400],
 [800, 1600, 2400],
 [800, 1008, 1344, 1600, 2400, 2864],
 [800, 1600, 2400],
 [800, 1600, 2256, 2416],
 [800, 1296, 1600, 1824, 2400],
 [400, 800, 1584, 1776, 1936, 2112, 2400],
 [608, 800, 1440, 1616, 2384, 2704],
 [800, 1136, 1600, 2400, 2752],
 [448, 800, 1600, 1888, 2384, 3024],
 [624, 800, 1600, 2032, 2384, 2672],
 [800, 1296, 1568, 2400, 2640, 2928],
 [784, 1264, 1600, 1808, 2240, 2400],
 [800, 1600, 2400, 2640],
 [800, 1600, 2400],
 [320, 800, 1600, 2400],
 [576, 800, 1600, 2176, 2400, 2896],
 [800, 1600, 2400],
 [128, 784, 1408, 1600, 2400],
 [800, 1600, 1856, 2384],
 [816, 1568, 2384],
 [800, 1600, 1888, 2400],
 [800, 1600, 2400],
 [256, 448, 800, 1600, 2400],
 [

In [11]:
import pickle
with open(path + 'wbs_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, loc_error_wbs, run_time_wbs, K_wbs], f)